# 基于Mindspore构造线性回归的损失函数---MSE（均方误差）损失函数

本小节主要介绍构造线性回归的损失函数的设计，使用MSE（均方误差）损失函数作为讲解实例,并使用自定义数据进行测试。

## 1、实验目的
- 理解MSE（均方误差）损失函数的意义。
- 自定义损失函数MSEloss。

## 2、MSE（均方误差）损失函数原理介绍
MSE用于计算预测值与标签值之间的均方误差。公式如下：
$$
MSE =\frac1{n}\sum_{i=1}^n({\vec y_i - y_i})^2
$$

$\vec y_i$为预测值，$y_i$为标签值，$n$为样本的数量。
MSE范围为 [ 0,+∞），当预测值与真实值完全相同时为0，误差越大，该值越大。

## 3、实验环境
在动手进行实践之前，需要注意以下几点：
* 确保实验环境正确安装，包括安装MindSpore。安装过程：首先登录[MindSpore官网安装页面](https://www.mindspore.cn/install)，根据安装指南下载安装包及查询相关文档。同时，官网环境安装也可以按下表说明找到对应环境搭建文档链接，根据环境搭建手册配置对应的实验环境。
* 推荐使用交互式的计算环境Jupyter Notebook，其交互性强，易于可视化，适合频繁修改的数据分析实验环境。
* 实验也可以在华为云一站式的AI开发平台ModelArts上完成。
* 推荐实验环境：MindSpore版本=MindSpore 2.0；Python环境=3.7


|  硬件平台 |  操作系统  | 软件环境 | 开发环境 | 环境搭建链接 |
| :-----:| :----: | :----: |:----:   |:----:   |
| CPU | Windows-x64 | MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.1节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| GPU CUDA 10.1|Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.2节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| Ascend 910  | Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第四章](./MindSpore环境搭建实验手册.docx)|

## 4、数据处理

### 4.1 数据准备

本实验实验使用根据公式$$z=a x^{2}+b y^{3} + c + noise$$
生成并添加噪声的数据，$a$,$b$,$c$为系数默认值分别为2,3,5；$noise$为噪声由均值为0方差为0.03的正态分布随机生成；$z$为标签值。数据形式为：$([x^{2},y^{3}],z)$，因为$x$,$y$,$noise$均需随机生成故此处不在展示数据。

### 4.2 数据加载
这里进行自定义数据集的生成和增强操作，首先`get_data`函数根据公式获取数据，然后在`create_dataset`函数生成数据集，并进行数据增强和处理操作：
- 定义进行数据增强和处理所需要的一些参数。
- 根据参数，生成对应的数据增强操作。
- 对生成的数据集进行处理。
- 对处理好的数据进行样例展示。


In [22]:
import numpy as np
from mindspore import dataset as ds

# 获取数据 num要生成的点数
def get_data(num, a=2.0, b=3.0, c=5.0):
    
    for _ in range(num):
        # 均匀分布
        x = np.random.uniform(-1.0, 1.0)
        y = np.random.uniform(-1.0, 1.0)
        # 添加噪声
        noise = np.random.normal(0, 0.03)
        z = a * x ** 2 + b * y ** 3 + c + noise
        yield np.array([[x**2], [y**3]],dtype=np.float32).reshape(1,2), np.array([z]).astype(np.float32)

# 生成数据集并增强
def create_dataset(num_data, batch_size=16, repeat_size=1):
    # 生成数据集
    input_data = ds.GeneratorDataset(list(get_data(num_data)), column_names=['xy','z'])
    # 划分批次
    input_data = input_data.batch(batch_size)
    # 增强数据集
    input_data = input_data.repeat(repeat_size)
    return input_data
 
data_number = 160       # 数据集大小
batch_number = 10       # 批量大小  
repeat_number = 10      # 增强次数

# 训练集
ds_train = create_dataset(data_number, batch_size=batch_number, repeat_size=repeat_number)
# 测试集
ds_test = create_dataset(data_number, batch_size=batch_number, repeat_size=repeat_number)

## 5、模型构建

模型的构建包括如下部分：
- 导入库和函数
- 定义线性模型
- 自定义损失函数MSEloss

### 5.1 导入所需库和函数并配置运行信息

In [23]:
# 时间处理模块
import time
# 科学计算库
import numpy as np
# MindSpore库
import mindspore as ms
# 常见算子操作
import mindspore.ops as ops
# 数据集处理模块
from mindspore import dataset as ds
# 环境设置模块，神经网络模块，张量，模型编译
from mindspore import nn, Tensor, Model
# 模型训练设置
from mindspore.train import Callback, LossMonitor
# L1型损失函数
from mindspore.nn import L1Loss
# MindSpore环境设置的0号种子
ms.common.set_seed(0)

### 5.2 定义线性模型

为了使用定义的损失函数（下一小节定义），通过基础nn.Cell类，构建了一个简单的线性模型，该模型只有一层全连接层（2输入节点，1输入），且只有前向传播。

In [24]:
# 定义线性模型
class LinearNet(nn.Cell):
    
    def __init__(self):
        super(LinearNet, self).__init__()
        # 全连接层
        self.fc = nn.Dense(2, 1, 0.02, 0.02)
 
    def construct(self, x):
        # 前向传播
        x = self.fc(x)
        return x

查看模型中参数维度

In [25]:
start_time = time.time()
net = LinearNet()
model_params = net.trainable_params()

# 显示模型的参数及其大小
print ('Param Shape is: {}'.format(len(model_params)))
for net_param in net.trainable_params():
    print(net_param, net_param.asnumpy())

Param Shape is: 2
Parameter (name=fc.weight, shape=(1, 2), dtype=Float32, requires_grad=True) [[0.02 0.02]]
Parameter (name=fc.bias, shape=(1,), dtype=Float32, requires_grad=True) [0.02]


### 5.3 自定义损失函数MSEloss

通过继承nn.LossBase基类来构建自己的损失函数，调用父类的初始化函数完成初始化，reduction参数指定求loss的均值，定义square参数为ops模块的Square()。

In [26]:
class MSELoss(nn.LossBase):  
    # """自定义损失函数MSEloss"""
    
    def __init__(self,reduction="mean"):
        # """完成初始化并求loss的均值"""
        
        super(MSELoss, self).__init__(reduction)
        self.square = ops.Square()    # """求平方算子"""
    
    def construct(self, base, target):
        x = self.square(base - target)   # """求平方运算"""
        return  self.get_loss(x)   # """返回loss的均值"""
        
user_loss = MSELoss()

## 6、模型训练

完成数据预处理、网络定义、损失函数之后，选择优化器，开始模型训练。模型训练包含1层迭代，数据集按分组从训练集中抽取数据，输入网络计算得到损失。


In [27]:
# 选择动量优化器
optim = nn.Momentum(net.trainable_params(), learning_rate=0.001, momentum=0.6)
# 使用Model接口将网络、损失函数和优化器关联起来
model = Model(net, user_loss, optim)
 
# 开始训练
epoch = 1
model.train(epoch, ds_train, callbacks=[LossMonitor(10)], dataset_sink_mode=True)
 
# 显示模型参数
for net_param in net.trainable_params():
    print(net_param, net_param.asnumpy())

# 显示训练时间
print ('The total time cost is: {}s'.format(time.time() - start_time))

epoch: 1 step: 10, loss is 22.83715057373047
epoch: 1 step: 20, loss is 19.59741973876953
epoch: 1 step: 30, loss is 27.788394927978516
epoch: 1 step: 40, loss is 22.2785701751709
epoch: 1 step: 50, loss is 24.237262725830078
epoch: 1 step: 60, loss is 15.516032218933105
epoch: 1 step: 70, loss is 18.288867950439453
epoch: 1 step: 80, loss is 11.845349311828613
epoch: 1 step: 90, loss is 13.60850715637207
epoch: 1 step: 100, loss is 8.196069717407227
epoch: 1 step: 110, loss is 12.773334503173828
epoch: 1 step: 120, loss is 10.078718185424805
epoch: 1 step: 130, loss is 7.707064628601074
epoch: 1 step: 140, loss is 15.063692092895508
epoch: 1 step: 150, loss is 4.523362159729004
epoch: 1 step: 160, loss is 7.742453098297119
Parameter (name=fc.weight, shape=(1, 2), dtype=Float32, requires_grad=True) [[ 1.0008343  -0.04824503]]
Parameter (name=fc.bias, shape=(1,), dtype=Float32, requires_grad=True) [2.9592104]
The total time cost is: 5.019861459732056s


## 7、模型预测

模型训练完成后，输入测试集进行模型预测，计算平方损失函数值。模型预测时不需要设置优化器，只需要设置网络、损失函数和评估指标。

In [21]:
# 模型预测
model = Model(net, loss_fn=user_loss, optimizer=None, metrics={'loss'})
# 计算测试集的平方损失函数值
pred_loss = model.eval(ds_test, dataset_sink_mode=False)
print(f'prediction loss is {pred_loss}')

prediction loss is {'loss': 7.63243814855814}
